In [1]:
import tvm
import tvm.relay as relay
import onnx
import numpy as np
from tvm.contrib import graph_executor

onnx_model = onnx.load("model.onnx")

mod, params = relay.frontend.from_onnx(onnx_model, shape={"args_0": (1, 784)})


mod = relay.transform.FoldConstant()(mod)  
mod = relay.transform.EliminateCommonSubexpr()(mod)  
mod = relay.transform.SimplifyInference()(mod)  
mod = relay.transform.CombineParallelDense()(mod)  

print("Updated Relay Intermediate Representation (IR):")
print(mod)

target = "llvm"  
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

lib.export_library("optimized_model_tvm.dll")

dev = tvm.cpu()  
module = graph_executor.GraphModule(lib["default"](dev))

dummy_input = np.random.rand(1, 784).astype("float32")  
module.set_input("args_0", dummy_input)  
module.run()

output = module.get_output(0).asnumpy()
print("TVM model output:", output)


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


Updated Relay Intermediate Representation (IR):
def @main(%args_0: Tensor[(1, 784), float32] /* ty=Tensor[(1, 784), float32] span=model/dense/MatMul.args_0:0:0 */) -> Tensor[(1, 10), float32] {
  %0 = nn.dense(%args_0, meta[relay.Constant][0] /* ty=Tensor[(128, 784), float32] span=model/dense/MatMul:0:0 */, units=None, out_dtype="float32") /* ty=Tensor[(1, 128), float32] span=model/dense/MatMul:0:0 */;
  %1 = add(%0, meta[relay.Constant][1] /* ty=Tensor[(128), float32] span=model/dense/BiasAdd.model/dense/BiasAdd/ReadVariableOp:0:0:0 */) /* ty=Tensor[(1, 128), float32] span=model/dense/BiasAdd:0:0 */;
  %2 = nn.relu(%1) /* ty=Tensor[(1, 128), float32] span=model/dense/Relu:0:0 */;
  %3 = nn.dense(%2, meta[relay.Constant][2] /* ty=Tensor[(10, 128), float32] span=model/dense_1/MatMul:0:0 */, units=None, out_dtype="float32") /* ty=Tensor[(1, 10), float32] span=model/dense_1/MatMul:0:0 */;
  %4 = add(%3, meta[relay.Constant][3] /* ty=Tensor[(10), float32] span=model/dense_1/BiasAdd.model/d